# Transports méridiens de chaleur et de quantité de mouvement

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Les fichiers de données au format netcdf (moyennes quotidiennes NCEP/NCAR) doivent être récupérés par FTP (FileZilla) et placés dans le répertoire data :

- Température : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/air.XXXX.nc
- Vent zonal : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/uwnd.XXXX.nc
- Vent méridien : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/vwnd.XXXX.nc

Il faut télécharger au moins la période 2000-2005 et ensuite concaténer les fichier netcdf avec les commandes NCO ncrcat ou CDO mergetime :

- https://linux.die.net/man/1/ncrcat
- https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-930002.2.6

--> fichier t.nc, u.nc, v.nc

<div class="alert alert-warning">

<b>Transports méridiens de chaleur et de quantité de mouvement zonal total : </b>
$$[\overline{vT}], [\overline{uv}]$$    
    
<br>

<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les perturbations transitoires : </b>
$$[\overline{u'v'}], [\overline{v'T'}]$$

<br>
    
<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques : </b>
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$

<br>

<b>Transports méridiens de chaleur et de quantité de mouvement zonal par la circulation en moyenne zonale : </b>
$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$
        
</div>

In [ ]:
%matplotlib inline

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from netCDF4 import Dataset
import h5py

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

# Traitement des données

In [ ]:
year1='2015'
year2='2020'
seasons=['DJF','JJA','MAM','SON']
plev1=1000
plev2=100
iplev1 = list(lev).index(plev1)
iplev2 = list(lev).index(plev2)

In [ ]:
diri="./daily_ncep/"

ft    = xr.open_dataset(diri+"t.nc").sel(time=slice(year1,year2)).sel(level=slice(plev1,plev2))
fu    = xr.open_dataset(diri+"u.nc").sel(time=slice(year1,year2)).sel(level=slice(plev1,plev2))
fv    = xr.open_dataset(diri+"v.nc").sel(time=slice(year1,year2)).sel(level=slice(plev1,plev2))

lat  = fv.lat.values
lon  = fv.lon.values
lev = fv.level.values

print(fv)
print(lev)

u = fu['uwnd']
v = fv['vwnd']
T = ft['air']

# Fonctions pour les moyennes et les anomalies

Moyenne temporelle : $$ \overline{\alpha} = \frac{1}{T} \int_0^T \alpha dt $$

Moyenne zonale : $$ \left[ \alpha \right] = \frac{1}{2\pi} \int_0^{2\pi} \alpha d\lambda $$

Anomalie temporelle : $$ \alpha^\prime = \alpha - \overline{\alpha} $$

Anomalie zonale : $$ \alpha^* = \alpha - \left[ \alpha \right] $$

In [ ]:
def bar(data, interval=fu.time.dt.season):
    return data.groupby(interval).mean(skipna=True)

def bracket(data):
    return data.mean(dim='lon', skipna=True)

def prime(data, interval=fu.time.dt.season):
    return data.groupby(interval) - bar(data, interval=interval)

def star(data):
    return data - bracket(data)

def global_mean(data):
    return data.weighted(np.cos(np.deg2rad(data.lat))).mean(dim=('lat','lon'), skipna=True)

# Flux méridiens de chaleur et de qdm zonal

<div class="alert alert-danger">
  
<b>Calculer les flux méridiens de chaleur et de quantité de mouvement zonal : </b>
    
<br>

Transports méridiens de chaleur et de quantité de mouvement zonal total :
$$[\overline{vT}], [\overline{uv}]$$
    
<br>

Transports méridiens de chaleur et de quantité de mouvement zonal par les perturbations transitoires :
$$[\overline{u'v'}], [\overline{v'T'}]$$

<br>
    
Transports méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques :
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$

<br>
    
Transports méridiens de chaleur et de quantité de mouvement zonal par la circulation en moyenne zonale :
$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$
    
</div>

In [ ]:
vT = bracket(bar(v*T))
vT_transient = bracket(bar(prime(v)*prime(T)))
vT_stationary = bracket(star(bar(v)) * star(bar(T)))
#vT_mmc = bracket(bar(v)) * bracket(bar(T) - global_mean(bar(T)))
vT_mmc = bracket(bar(v)) * bracket(bar(T))

uv = bracket(bar(u*v))
uv_transient = bracket(bar(prime(u)*prime(v)))
uv_stationary = bracket(star(bar(u)) * star(bar(v)))
#uv_mmc = bracket(bar(v)) * bracket(bar(u) - global_mean(bar(u)))
uv_mmc = bracket(bar(v)) * bracket(bar(u))

print(vT_transient.shape)

In [ ]:
scale=np.cos(lat[np.newaxis,np.newaxis,:]*np.pi/180.)
vT=vT*scale
vT_transient=vT_transient*scale
vT_stationary=vT_stationary*scale
vT_mmc=vT_mmc*scale
uv=uv*scale
uv_transient=uv_transient*scale
uv_stationary=uv_stationary*scale
uv_mmc=uv_mmc*scale

# Flux méridiens par les transitoires

In [ ]:
levels_vT = np.arange(-15,17,2)
levels_vT = np.arange(-7,8,1)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v'T'}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v'T'}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_transient[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_transient[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v'T'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_transient.mean('season').mean('level'), label='Annual')
plt.plot(lat, vT_transient[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_transient[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/eddy_heat_mean'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-50,55,5)
levels_uv = np.arange(-24,26,2)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u'v'}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u'v'}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_transient[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_transient[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentum_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport ($m^2/s^2$)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u'v'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_transient.mean('season').mean('level'), label='Annual')
plt.plot(lat, uv_transient[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_transient[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/eddy_momentum_mean'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Flux méridiens par les circulations stationnaires non axisymétriques

In [ ]:
levels_vT = np.arange(-15,17,2)
levels_vT = np.arange(-7,8,1)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v^*}\overline{T^*}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v^*}\overline{T^*}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_stationary[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_stationary[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/stat_heat_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v^*}\overline{T^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_stationary.mean('season').mean('level'), label='Annual')
plt.plot(lat, vT_stationary[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_stationary[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/stat_heat_mean'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-50,55,5)
levels_uv = np.arange(-24,26,2)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u^*}\overline{v^*}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentum_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u^*}\overline{v^*}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_stationary[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_stationary[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/stat_momentum_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('$Transport (m^2/s^2)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u^*}\overline{v^*}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_stationary.mean('season').mean('level'), label='Annual')
plt.plot(lat, uv_stationary[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_stationary[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/stat_momentum_flux_mean'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Flux méridiens par les circulations stationnaires axisymétriques

$$\overline{[v]} \overline{[T]}, \overline{[u]} \overline{[v]}$$

In [ ]:
levels_vT = np.arange(-600,700,100)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$\overline{[v]} \overline{[T]}$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc.mean('season'), levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc.mean('season'), levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_heat_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$\overline{[v]} \overline{[T]}$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc[0,:,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc[0,:,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vT_mmc[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vT_mmc[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_heat_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$\overline{[v]} \overline{[T]}$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vT_mmc.mean('season').mean('level'), label='Annual')
plt.plot(lat, vT_mmc[0,:].mean('level'), label='DJF')
plt.plot(lat, vT_mmc[1,:].mean('level'), label='JJA')
plt.legend()

figname='./figs/zmean_heat_mean'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-50,55,5)
levels_uv = np.arange(-24,26,2)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$\overline{[u]} \overline{[v]}$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc.mean('season'), levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc.mean('season'), levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/zmean_momentum_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$\overline{[u]} \overline{[v]}$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc[0,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uv_mmc[1,:,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uv_mmc[1,:,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/zmean_momentum_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('$Transport (m^2/s^2)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$\overline{[u]} \overline{[v]}$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uv_mmc.mean('season').mean('level'), label='Annual')
plt.plot(lat, uv_mmc[0,:].mean('level'), label='DJF')
plt.plot(lat, uv_mmc[1,:].mean('level'), label='JJA')

plt.legend()

figname='./figs/zmean_momentum_flux'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()